In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import sys
sys.modules['Image'] = Image 
import pandas as pd
import numpy as np
import cv2
import os

In [34]:
# Globals   
path_to_data_dir = "../../database/sorted_by_state/"
test_file = "Arkansas/_09TL3yHwqMzcB8m4BUGoQ.jpeg"

In [35]:
data = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [54]:
# train_dataset = data.flow_from_directory(path_to_data_dir, target_size=(224, 224), batch_size=64, class_mode='categorical', shuffle=True, subset='training')
# validation_dataset = data.flow_from_directory(path_to_data_dir, target_size=(224, 224), batch_size=64, class_mode='categorical', shuffle=False, subset="validation")
train_dataset, validation_dataset = tf.keras.utils.image_dataset_from_directory(path_to_data_dir, labels='inferred', batch_size = 64, shuffle = True, image_size=(224, 224), validation_split=0.2, subset='both', seed=42)

Found 51175 files belonging to 54 classes.
Using 40940 files for training.
Using 10235 files for validation.


In [ ]:
# model with 3 convolution layers max pooled 3 times with relu as activation function and softmax applied to output layer
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=None, padding='valid'),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), activation='relu', use_bias=True),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=None, padding='valid'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu', use_bias=True),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=None, padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(54, activation='softmax')
])


model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=RMSprop(learning_rate=0.001),
              metrics=[
              tf.keras.metrics.Accuracy(),
              tf.keras.metrics.Recall(),
              tf.keras.metrics.Precision(),
              tf.keras.metrics.F1Score(),
            ],
              )

In [ ]:
from tensorflow.keras.callbacks import Callback

epochs_to_save = [1,2]
model_version= "v1.0"

save_dir = "saved_models/"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

class CustomClallback(Callback):
    def __init__(self, epochs_to_save):
        super().__init__()
        self.epochs_to_save = epochs_to_save

    def on_epoch_end(self, epoch, logs=None):
        if epoch in self.epochs_to_save:
            fn = f'{model_version}_{epoch:02d}.keras'

            # Save the model weights
            self.model.save(save_dir + fn)
            print(f"Model weights saved at epoch {epoch} to {fn}")


custom_callback = CustomClallback(epochs_to_save=epochs_to_save)


model_fit = model.fit(train_dataset, 
                    # epochs=epochs_to_save[-1] + 1, #+1 to include the last epoch
                    validation_data=validation_dataset,
                    # callbacks=[custom_callback],
                    )

for epoch in epochs_to_save:
    fn = f'{model_version}_{epoch:02d}.keras'
    model.load_weights(save_dir + fn)

    # Evaluate the model on the validation dataset
    loss, accuracy, recall, precision, f1_score = model.evaluate(validation_dataset)
    print(f"Model {fn} - Loss: {loss}, Validation Accuracy: {accuracy}, Validation Precision: {precision}, Validation Recall: {recall}, Validation f1_score: {f1_score}")

ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 54)